In [1]:
from datasets import AdjacentDuplicate
import pandas as pd
import torch
import numpy as np

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = AdjacentDuplicate(num_points=1000)

In [3]:
# 0th index is data, 1xt index is label
data.train_data.dataset.tensors[0][:10], data.train_data.dataset.tensors[1][:10]

(tensor([[11,  8, 15, 17,  4, 11, 13,  7, 18, 16],
         [13, 13,  9, 18, 16,  1, 15,  7, 16, 19],
         [ 5, 15, 14, 18, 16, 17,  9, 14, 17,  3],
         [15, 16,  8,  7, 11, 19, 17,  7, 11, 18],
         [ 6, 19,  4, 12, 13, 10,  4, 17,  3,  7],
         [15,  6, 16,  2,  7,  8, 13, 11,  9,  2],
         [14,  4, 11,  4,  6,  6,  2,  8,  8, 19],
         [ 9,  2, 15,  4,  6, 14,  1,  8, 16,  3],
         [ 8,  2,  6,  6, 18, 16,  8, 17, 18,  7],
         [ 9,  1,  4,  1, 15, 11,  4, 15,  2, 13]]),
 tensor([0, 1, 0, 0, 0, 0, 1, 0, 1, 0]))

In [4]:
def view_class_balance(dataset):
    return pd.Series(next(iter(torch.utils.data.DataLoader(dataset, batch_size=len(data.train_data))))[1]).value_counts()

print("Train data:\n", view_class_balance(data.train_data))
print("Val data:\n", view_class_balance(data.val_data))
print("Test data:\n", view_class_balance(data.test_data))

Train data:
 1    358
0    342
dtype: int64
Val data:
 0    54
1    46
dtype: int64
Test data:
 0    104
1     96
dtype: int64


In [5]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
from transformers import BertConfig, BfBertForSequenceClassification
config = BertConfig.from_pretrained(pretrained_model_name_or_path="../../brunoflow/models/bert/config-toy.json")
m = BfBertForSequenceClassification(config=config)


env: XLA_PYTHON_CLIENT_PREALLOCATE=false


2023-03-24 02:10:31.966284: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:181: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  bf.Parameter(jnp.zeros(self.position_ids.shape, dtype=jnp.int64), name="position_ids"),


In [6]:
np.log(2)

0.6931471805599453

In [7]:
import brunoflow as bf
from jax import numpy as jnp 
x, y = next(iter(torch.utils.data.DataLoader(data.train_data, batch_size=128)))
# x
outputs = m(bf.Node(jnp.array(x.numpy())), labels=bf.Node(jnp.array(y.numpy())))
outputs.loss, outputs.logits

(node(name: /, val: 0.6931512355804443, grad: 0.0),
 node(name: +, val: [[-2.2405493e-03 -2.3315039e-03]
  [-2.2404727e-03 -2.3314159e-03]
  [ 2.4030702e-03  1.9078741e-03]
  [-3.1292469e-03 -3.1167686e-03]
  [ 2.4029738e-03  1.9077738e-03]
  [-1.2130328e-03 -2.2829117e-03]
  [-3.0820821e-03 -3.1103259e-03]
  [ 2.4032553e-03  1.9080776e-03]
  [-5.6432741e-04 -1.8224908e-03]
  [-6.4956426e-04 -5.6442287e-04]
  [-6.5026281e-04 -5.6515425e-04]
  [-6.4951606e-04 -5.6443969e-04]
  [-2.0050718e-03 -1.9435909e-03]
  [ 4.9756485e-04  9.9328987e-04]
  [ 2.2207994e-03  1.7125086e-03]
  [ 2.4032027e-03  1.9079753e-03]
  [-6.4953230e-04 -5.6449318e-04]
  [ 2.4029440e-03  1.9077680e-03]
  [-6.1749277e-04 -1.7200704e-03]
  [ 4.9799879e-04  9.9380116e-04]
  [-5.6543510e-04 -1.8232890e-03]
  [-2.3389147e-03 -9.2535524e-04]
  [-1.8889657e-03 -6.1867875e-04]
  [ 4.9964845e-04  9.9509803e-04]
  [-1.8901826e-03 -6.1988301e-04]
  [-2.0044891e-03 -1.9431220e-03]
  [-1.0780154e-03 -4.1020929e-04]
  [-6.49502